<a href="https://colab.research.google.com/github/pkse-searcher/fsvm-pois-attack-defense/blob/main/Attack_Project_Progress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn import svm
from random import randint, shuffle, uniform
from copy import copy,deepcopy
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from math import isclose, fabs
try:
  import secml
except:
  !pip install git+https://gitlab.com/secml/secml
  import secml
from secml.ml.classifiers import CClassifierSVM
from secml.ml.kernels import CKernelRBF
from secml.adv.attacks import CAttackPoisoningSVM
from secml.data import CDataset
from secml.data.splitter import CTrainTestSplit
from secml.ml.kernels.c_kernel_linear import CKernelLinear
from secml.data.loader import CDLRandomBlobs
from secml.array import CArray
# NBVAL_IGNORE_OUTPUT
from secml.data.loader import CDataLoaderMNIST, CDataLoaderCIFAR10



try:
  from pyod.models.knn import KNN
except:
  !pip install pyod
  from pyod.models.knn import KNN
from pyod.models.cblof import CBLOF
from pyod.models.hbos import HBOS
from pyod.models.copod import COPOD
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://gitlab.com/secml/secml to /tmp/pip-req-build-kj_idj3j
  Running command git clone -q https://gitlab.com/secml/secml /tmp/pip-req-build-kj_idj3j
  Created wheel for secml: filename=secml-0.15.dev0+fd905aa5-py3-none-any.whl size=4697385 sha256=9bec6e2b874949d6bed8e4bad3155cc2e02c6d46980f93e28a679323cd518347
  Stored in directory: /tmp/pip-ephem-wheel-cache-ncnru47y/wheels/84/fd/ae/3df0e5ded70566668585bd0f28118c0de075ff96260f3725cb
Successfully built secml
2022-10-06 13:20:51,575 - secml.settings - INFO - New `SECML_HOME_DIR` created: /root/secml-data
2022-10-06 13:20:51,575 - secml.settings - INFO - New `SECML_HOME_DIR` created: /root/secml-data


INFO:secml.settings:New `SECML_HOME_DIR` created: /root/secml-data


2022-10-06 13:20:51,592 - secml.settings - INFO - Default configuration file copied to: /root/secml-data/secml.conf
2022-10-06 13:20:51,592 - secml.settings - INFO - Default configuration file copied to: /root/secml-data/secml.conf


INFO:secml.settings:Default configuration file copied to: /root/secml-data/secml.conf


2022-10-06 13:20:51,604 - secml.settings - INFO - New `SECML_DS_DIR` created: /root/secml-data/datasets
2022-10-06 13:20:51,604 - secml.settings - INFO - New `SECML_DS_DIR` created: /root/secml-data/datasets


...

2022-10-06 13:20:51,636 - secml.settings - INFO - New `SECML_LOGS_DIR` created: /root/secml-data/logs
2022-10-06 13:20:51,636 - secml.settings - INFO - New `SECML_LOGS_DIR` created: /root/secml-data/logs


INFO:secml.settings:New `SECML_LOGS_DIR` created: /root/secml-data/logs


2022-10-06 13:20:51,642 - secml.settings - INFO - New `SECML_PYTORCH_DIR` created: /root/secml-data/pytorch-data
2022-10-06 13:20:51,642 - secml.settings - INFO - New `SECML_PYTORCH_DIR` created: /root/secml-data/pytorch-data


INFO:secml.settings:New `SECML_PYTORCH_DIR` created: /root/secml-data/pytorch-data


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 137 kB 5.4 MB/s 
  Created wheel for pyod: filename=pyod-1.0.5-py3-none-any.whl size=170258 sha256=f28418e06fa530cfc11ed0c32a78317967c536ea2331c38895790eeacf374632
  Stored in directory: /root/.cache/pip/wheels/29/98/25/bc9bfccf3907de246c61668bc09c39216ec157f8218aea4c16
Successfully built pyod


In [ ]:
#@title Default title text
dataset =  'MNIST' #'CIFAR10' #
ATTACK_TYPE = 'OUR_ATTACK_GD' # 'OUR_ATTACK' # 'RANDOM' # 'OPTIMAL' # 'LABEL_FLIP' # 'GRADIENT' #
MODE = 'COLLABORAT' # 'INDIV' #
num_local_machine = 100
print(dataset, ATTACK_TYPE, MODE, num_local_machine)


dataLoad()


for advNumber in [45]: # advNumber means how many attackers
  
  hSets = []
  for local_machine in range(num_local_machine):
    X_local, y_local= load_local_data(advNumber, local_machine)

    # print(local_machine, y_local)

    clfLocals = CClassifierSVM(C=1)
    clfLocals.fit(X_local, y_local)

    w, b = clfLocals.w.get_data()[0], clfLocals.b.get_data()[0]
    m = np.sqrt(w.dot(w))
    w = w / m
    b /= m

    w, b = model_change(advNumber, local_machine, w, b)
    hSets.append(np.append(w,b))


  outler_input = deepcopy(hSets)
  # Average
  w = np.mean(hSets, axis=0)

  cnt = 0
  for i in range(len(testX)):
    det = np.dot(np.append(testX[i], 1), w)
    if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1): 
      cnt += 1 
    # print(i, det, testy[i])
  print(f'Time {datetime.now()}: Accuracy is {cnt}/{len(testX)} = {cnt/len(testX)} with {advNumber} attacker after average')

    # Average
  w = hSets[0]

  cnt = 0
  for i in range(len(testX)):
    det = np.dot(np.append(testX[i], 1), w)
    if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1): 
      cnt += 1 
    # print(i, det, testy[i])
  print(f'Time {datetime.now()}: Accuracy is {cnt}/{len(testX)} = {cnt/len(testX)} with {advNumber} attacker poisonus hyperplane')

  
    # Average
  w = hSets[70]

  cnt = 0
  for i in range(len(testX)):
    det = np.dot(np.append(testX[i], 1), w)
    if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1): 
      cnt += 1 
    # print(i, det, testy[i])
  print(f'Time {datetime.now()}: Accuracy is {cnt}/{len(testX)} = {cnt/len(testX)} with {advNumber} attacker good hyperplane')



  # Clustering
  kmeans = KMeans(n_clusters=1, random_state=0).fit(hSets)
  w = kmeans.cluster_centers_[0]

  cnt = 0
  for i in range(len(testX)):
    det = np.dot(np.append(testX[i], 1), w)
    if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1): cnt += 1
    # print(det, testX[i], testy[i])
  print(f'Time {datetime.now()}: Accuracy is {cnt}/{len(testX)} = {cnt/len(testX)} with {advNumber} attacker after clustering')


  # Convergence
  isUpdated = True
  iter = 10000
  while isUpdated and iter > 0:
    iter -= 1
    isUpdated = False
    kmeans = KMeans(n_clusters=1, random_state=0).fit(hSets)
    w = kmeans.cluster_centers_[0]
    for i in range(num_local_machine):
      if not np.allclose(w, hSets[i]):
        hSets[i] = np.mean( np.array([ hSets[i], w ]), axis=0 )
        isUpdated = True

  kmeans = KMeans(n_clusters=1, random_state=0).fit(hSets)
  w = kmeans.cluster_centers_[0]
  cnt = 0
  for i in range(len(testX)):
    det = np.dot(np.append(testX[i], 1), w)
    if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1): cnt += 1
  print(f'Time {datetime.now()}: Accuracy is {cnt}/{len(testX)} = {cnt/len(testX)} with {advNumber} attacker after convergence')


 

MNIST OUR_ATTACK_GD COLLABORAT 100

File stored in `/root/secml-data/datasets/mnist/train-images-idx3-ubyte.gz`

File stored in `/root/secml-data/datasets/mnist/train-labels-idx1-ubyte.gz`

File stored in `/root/secml-data/datasets/mnist/t10k-images-idx3-ubyte.gz`

File stored in `/root/secml-data/datasets/mnist/t10k-labels-idx1-ubyte.gz`
Time 2022-10-06 13:21:12.522213: Accuracy is 977/1000 = 0.977 with 45 attacker after iter 0 and error 1553.8583236195416
Time 2022-10-06 13:21:22.811599: Accuracy is 59/1000 = 0.059 with 45 attacker after iter 1 and error 188.20290600509446
Time 2022-10-06 13:21:32.572358: Accuracy is 36/1000 = 0.036 with 45 attacker after iter 2 and error 113.80699255446532
Time 2022-10-06 13:21:38.755623: Accuracy is 27/1000 = 0.027 with 45 attacker after iter 3 and error 83.50272478236454
Time 2022-10-06 13:21:45.008582: Accuracy is 22/1000 = 0.022 with 45 attacker after iter 4 and error 68.11910940815878
Time 2022-10-06 13:21:51.300721: Accuracy is 20/1000 = 0.02 

In [ ]:
hSets

NameError: ignored

In [ ]:
def model_change(advNumber, local_machine, w, b):
    if local_machine>=advNumber: # if non attacker no model is changed
        return w, b
    global old_w, old_b
    if ATTACK_TYPE == 'RANDOM':
        if MODE == 'COLLABORAT' and local_machine != 0:
            return old_w, old_b
        tw = deepcopy(w)
        mx = w.max()
        mn = w.min()
        for x in range(w.size):
            tw[x] = uniform(mn, mx)
        old_w, old_b = tw, b
        return tw, b

    if ATTACK_TYPE == 'OUR_ATTACK':
        if MODE == 'COLLABORAT' and local_machine != 0:
            return old_w, old_b
        tw = deepcopy(w)
        mx = w.max()
        mn = w.min()
        # print(mn, mx)
        for j in range(w.size):
            sum = 0
            for i in range(len(XGlobal)):
                sum += (-1 if yGlobal[i] == 0 else 1) * XGlobal[i][j]
                # print('\t', i, j, -1 if testy[i] == 0 else 1, testX[i][j], sum)
            tw[j] = mn if sum >= 0 else mx
            # tw[j] = -fabs(w[j]) if sum >= 0 else fabs(w[j])
            # tw[j] = -1000 if sum >= 0 else 1000
            # print(j, w[j], tw[j], sum)
        m = np.sqrt(tw.dot(tw))
        # print(m)
        # tw = tw / m
        old_w, old_b = tw,b
        return tw, b

    if ATTACK_TYPE == 'OUR_ATTACK_GD':
        h=1e-5
        if MODE == 'COLLABORAT' and local_machine != 0:
            return old_w, old_b
        tw = deepcopy(w)
        def get_error(tw):
            w_g = (advNumber*tw + (num_local_machine-advNumber)*w)/num_local_machine
            error_sum=0
            for i in range(len(testX)):
                res = 1+(-1 if testy[i] == 0 else 1) * (np.dot(testX[i],w_g)+b)
                # print(f"res {res} y {yGlobal[i]} and det {np.dot(testX[i],w_g)+b}")
                error_sum += max(0, res)
            return error_sum
        iter=50
        prev_acc = 0
        for _ in range(iter):
            cur_acc = get_error(tw)
            if(abs(prev_acc-cur_acc)/cur_acc<1e-4):
                break
            prev_acc = cur_acc
            cnt = 0
            for i in range(len(testX)):
                det = np.dot(np.append(testX[i], 1), np.append(tw,b))
                if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1): cnt += 1
            print(f'Time {datetime.now()}: Accuracy is {cnt}/{len(testX)} = {cnt/len(testX)} with {advNumber} attacker after iter {_} and error {cur_acc} and ratio with previous {abs(prev_acc-cur_acc)/cur_acc}')



            for j in range(w.size):
                tw[j]=tw[j]+h
                err_add = get_error(tw)

                tw[j]=tw[j]-2*h
                err_sub = get_error(tw)

                tw[j]=tw[j]+h
                gradient = (err_add-err_sub)/(2*h)
                # print(err_add,err_sub, gradient)

                tw[j]=tw[j]-0.03*gradient
        old_w, old_b = tw,b
        return tw, b
        


In [ ]:

def load_local_data(advNumber, local_machine):
    global old_x_local, old_y_local
    
    if local_machine>=advNumber: # if non attacker no dataset is changed
        Xlocal = deepcopy(XGlobal[local_machine * slc : (local_machine + 1) * slc])
        ylocal = deepcopy(yGlobal[local_machine * slc : (local_machine + 1) * slc])
        return Xlocal, ylocal
    
    
    if MODE=="COLLABORAT":
        if local_machine !=0:
            return old_x_local, old_y_local
        Xlocal = deepcopy(XGlobal[0 : advNumber * slc])
        ylocal = deepcopy(yGlobal[0 : advNumber * slc])
    else:
        Xlocal = deepcopy(XGlobal[local_machine * slc : (local_machine + 1) * slc])
        ylocal = deepcopy(yGlobal[local_machine * slc : (local_machine + 1) * slc])

    
    if ATTACK_TYPE == 'RANDOM' or ATTACK_TYPE == 'OUR_ATTACK' or ATTACK_TYPE == 'OUR_ATTACK_GD': # if model attacker then no dataset is changed
        old_x_local, old_y_local = Xlocal, ylocal
        return Xlocal, ylocal

    
    if ATTACK_TYPE == 'LABEL_FLIP' or ATTACK_TYPE == 'OUR_ATTACK':
        for j in range(len(ylocal)):
            ylocal[j] ^= 1
        old_x_local, old_y_local = Xlocal, ylocal
        return Xlocal, ylocal

    if ATTACK_TYPE == 'OPTIMAL':
        n_tr = int(len(ylocal)*0.4)  # Number of training set samples
        n_val = int(len(ylocal)*0.5)  # Number of validation set samples
        n_ts = len(ylocal) - n_tr - n_val  # Number of test set samples

        # Split in training, validation and test
        dataset = CDataset(Xlocal, ylocal)
        random_state = 999
        splitter = CTrainTestSplit(train_size=n_tr + n_val, test_size=n_ts, random_state=random_state)
        tr_val, ts = splitter.split(dataset)
        splitter = CTrainTestSplit(train_size=n_tr, test_size=n_val, random_state=random_state)
        tr, val = splitter.split(dataset)
        tr = tr.append(ts)

        clf = CClassifierSVM(kernel=CKernelLinear(), C=1)

        # We can now fit the classifier
        clf.fit(tr.X, tr.Y)

        # Should be chosen depending on the optimization problem
        lb, ub = 0., 1.  # Bounds of the attack space. Can be set to `None` for unbounded

        # Should be chosen depending on the optimization problem
        solver_params = {
            'eta': 0.25,
            'eta_min': 2.0,
            'eta_max': None,
            'max_iter': 200,
            'eps': 1e-8
        }

        pois_attack = CAttackPoisoningSVM(classifier=clf,
                                      training_data=tr,
                                      val=val,
                                      lb=lb, ub=ub,
                                      solver_params=solver_params,
                                      random_seed=random_state)

        n_poisoning_points = 10  # Number of poisoning points to generate
        pois_attack.n_points = n_poisoning_points

        #Run the poisoning attack
        # print("Attack started...")
        pois_y_pred, pois_scores, pois_ds, f_opt = pois_attack.run(ts.X, ts.Y)
        # print("Attack complete!")
        # tr = tr.append(pois_ds) #.append(val)
        Xlocal, ylocal = pois_ds.X.get_data(), pois_ds.Y.get_data()
        old_x_local, old_y_local = Xlocal, ylocal
        # print(Xlocal)
        # print(ylocal)
        return Xlocal, ylocal

    
    
    
    


In [ ]:
def dataLoad():
    global dataset, XGlobal, yGlobal, testX, testy, slc
    if dataset == 'MNIST':
        # MNIST dataset will be downloaded and cached if needed
        loader = CDataLoaderMNIST()

        slc = 100
        size_all = 784

        # total MNIST data with 5 and 9 is 11370
        n_samples = slc * num_local_machine  # Number of samples
        digits = (5, 9)

        dataset = loader.load('training', digits=digits)
        # Normalize the features in `[0, 1]`
        dataset.X /= 255
        XGlobal, yGlobal = dataset.X.get_data(), dataset.Y.get_data()

        test_dataset = loader.load('testing', digits=digits, num_samples=1000)
        test_dataset.X /= 255
        testX, testy = test_dataset.X.get_data(), test_dataset.Y.get_data()

    elif dataset == 'CIFAR10':
        # CIFAR10 dataset will be downloaded and cached if needed
        loader = CDataLoaderCIFAR10()

        def rgb_2_gray(arr):
            return [0.2989 * arr[i] + 0.5870 * arr[i+1024] + 0.1140 * arr[i+2048] for i in range(1024)]

        size_all = 1024
        slc = 100

        # total MNIST data with 5 and 9 is 11370
        n_samples = slc * num_local_machine  # Number of samples

        dataset, test_dataset = loader.load()
        # Normalize the features in `[0, 1]`
        dataset.X /= 255

        XGlobal, yGlobal = [], []
        trainX = dataset.X.get_data()
        trainY = dataset.Y.get_data()

        for i in range(len(trainY)):
            if trainY[i] < 2:
                XGlobal.append(rgb_2_gray(trainX[i]))
                yGlobal.append(trainY[i])

        unique, counts = np.unique(yGlobal, return_counts=True)
        print(f"Global count {dict(zip(unique, counts))}")
        testX, testy = [], []
        test_dataset.X /= 255
        dataX = test_dataset.X.get_data()
        dataY = test_dataset.Y.get_data()

        print(len(trainY), len(dataY))

        for i in range(len(dataY)):
            if dataY[i] < 2:
                testX.append(rgb_2_gray(dataX[i]))
                testy.append(dataY[i])

        unique, counts = np.unique(testy, return_counts=True)
        print(f"Test count {dict(zip(unique, counts))}")
        print(len(yGlobal), len(testy))